In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf


/home/rodo/code/proyecto_4/004_DL/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-02 19:04:45.993222: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-02 19:04:45.993470: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 19:04:45.995390: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-02 19:04:46.019603: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instru

In [2]:
class TradingStrategyOptimizer:
    def __init__(self, buy_data_path, sell_data_path):
        self.buy_data_path = buy_data_path
        self.sell_data_path = sell_data_path
        
    
    def load_and_preprocess_data(self, path):
        data = pd.read_csv(path)
        y = data.pop('Y_BUY' if 'buy' in path else 'Y_SELL').values
        X = data.values
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        return train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    def objective(self, trial, X_train, y_train, X_val, y_val):
        n_layers = trial.suggest_int('n_layers', 1, 3)
        n_units = trial.suggest_int('n_units', 50, 200)
        lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
        
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(n_units, input_dim=X_train.shape[1], activation='relu'))
        for _ in range(n_layers - 1):
            model.add(tf.keras.layers.Dense(n_units, activation='relu'))
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])
        
        model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, verbose=False)
        _, accuracy = model.evaluate(X_val, y_val, verbose=False)
        return accuracy
    
    def optimize_dnn(self, X_train, y_train, X_val, y_val):
        study = optuna.create_study(direction='maximize')
        func = lambda trial: self.objective(trial, X_train, y_train, X_val, y_val)
        study.optimize(func, n_trials=5)
        return study.best_trial.params

    def build_and_train_model(self, best_params, X_train, y_train):
        n_layers = best_params['n_layers']
        n_units = best_params['n_units']
        lr = best_params['lr']
        
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(n_units, input_dim=X_train.shape[1], activation='relu'))
        for _ in range(n_layers - 1):
            model.add(tf.keras.layers.Dense(n_units, activation='relu'))
        model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr), 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])
        
        model.fit(X_train, y_train, epochs=5, verbose=False)
        return model
    
    # create a dataframe with the Y_buy and Y_sell columns predicted
    def run(self):
        X_train_buy, X_val_buy, y_train_buy, y_val_buy = self.load_and_preprocess_data(self.buy_data_path)
        X_train_sell, X_val_sell, y_train_sell, y_val_sell = self.load_and_preprocess_data(self.sell_data_path)
        
        best_params_buy = self.optimize_dnn(X_train_buy, y_train_buy, X_val_buy, y_val_buy)
        best_params_sell = self.optimize_dnn(X_train_sell, y_train_sell, X_val_sell, y_val_sell)
        
        model_buy = self.build_and_train_model(best_params_buy, X_train_buy, y_train_buy)
        model_sell = self.build_and_train_model(best_params_sell, X_train_sell, y_train_sell)
        
        y_pred_buy = model_buy.predict(X_val_buy)
        y_pred_sell = model_sell.predict(X_val_sell)
        
        # covertir a booleanos
        y_pred_buy = y_pred_buy > 0.5
        y_pred_sell = y_pred_sell > 0.5
        
        
        return pd.DataFrame({'Y_BUY_PRED': y_pred_buy.flatten(), 'Y_SELL_PRED': y_pred_sell.flatten()})

In [4]:
if __name__ == '__main__':
    optimizer = TradingStrategyOptimizer('/home/rodo/code/proyecto_4/004_DL/data/close_data_buy_5.csv', '/home/rodo/code/proyecto_4/004_DL/data/close_data_sell_5.csv')
    df = optimizer.run()
    print(df)

[I 2024-04-02 19:04:47,021] A new study created in memory with name: no-name-19724f0a-0fea-4f72-8250-f8230c32736a
/tmp/ipykernel_31072/2717340414.py:18: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/home/rodo/code/proyecto_4/004_DL/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-04-02 19:04:48.674392: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.

248/248 ━━━━━━━━━━━━━━━━━━━━ 0s 587us/step
248/248 ━━━━━━━━━━━━━━━━━━━━ 0s 483us/step
      Y_BUY_PRED  Y_SELL_PRED
0          False         True
1           True         True
2           True        False
3           True         True
4           True         True
...          ...          ...
7903        True        False
7904        True        False
7905        True         True
7906        True        False
7907        True        False

[7908 rows x 2 columns]
